In [ ]:
import cv2
from ultralytics import YOLO
import os

def run_yolo_on_video(model_path, video_path, output_path=None, conf_threshold=0.5):
    model = YOLO(model_path)

    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"Video: {width}x{height} @ {fps}fps, {total_frames} frames")

    out = None
    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        if not out.isOpened():
            print(f"Error: Could not open video writer for {output_path}")
            out = None


    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % 100 == 0 or frame_count == 1 or frame_count == total_frames:
             print(f"Processing frame {frame_count}/{total_frames}")

        results = model(frame, conf=conf_threshold, verbose=False)

        for result in results:
            boxes = result.boxes

            if boxes is not None:
                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    confidence = box.conf[0].cpu().numpy()
                    class_id = int(box.cls[0].cpu().numpy())

                    class_name = model.names[class_id]

                    color = (0, 255, 0)
                    thickness = 2
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness)

                    label = f"{class_name}: {confidence:.2f}"
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    font_scale = 0.5
                    text_thickness = 1
                    text_x = int(x1)
                    text_y = int(y1) - 10
                    if text_y < 10:
                         text_y = int(y1) + 20

                    cv2.putText(frame, label, (text_x, text_y),
                               font, font_scale, color, text_thickness)
        if out is not None:
            out.write(frame)
    cap.release()
    if out is not None:
        out.release()
    print(f"Finished processing. Processed {frame_count} frames.")
    if output_path and out is not None:
        print(f"Output video saved to {output_path}")
    elif output_path and out is None:
         print(f"Failed to save output video to {output_path}")


if __name__ == "__main__":
    MODEL_PATH = "/content/best.pt"
    VIDEO_PATH = "/content/broadcast.mp4"
    OUTPUT_PATH = "/content/broadcast_detections.mp4"

    if not os.path.exists(VIDEO_PATH):
        print(f"Error: Video file not found at {VIDEO_PATH}")
    elif not os.path.exists(MODEL_PATH):
         print(f"Error: Model file not found at {MODEL_PATH}")
    else:
        run_yolo_on_video(
            model_path=MODEL_PATH,
            video_path=VIDEO_PATH,
            output_path=OUTPUT_PATH,
            conf_threshold=0.5
        )

Video: 1920x1080 @ 24fps, 132 frames
Processing frame 1/132
Processing frame 100/132
Processing frame 132/132
Finished processing. Processed 132 frames.
Output video saved to /content/broadcast_detections.mp4


In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [9]:
bytetrack_config = """
tracker_type: bytetrack
track_high_thresh: 0.4
track_low_thresh: 0.1
new_track_thresh: 0.5
track_buffer: 600   # Increased from default (30)
match_thresh: 0.8
fuse_score: True # Added fuse_score parameter
"""
with open('/content/bytetrack_custom.yaml', 'w') as f:
    f.write(bytetrack_config)

In [10]:
import cv2
from ultralytics import YOLO

# Load your fine-tuned YOLOv11 model
model = YOLO('/content/best.pt')  # Replace with your weights

# Open the video file
video_path = '/content/15sec_input_720p.mp4'
cap = cv2.VideoCapture(video_path)

# Set up video writer for output (optional)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('/content/output_tracking.mp4', fourcc, 30.0,
                      (int(cap.get(3)), int(cap.get(4))))

# Trackers will handle ID assignment
tracker_config = '/content/bytetrack_custom.yaml'  # Or 'botsort.yaml'

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv11 detection + ByteTrack tracking
    results = model.track(
        frame,
        tracker=tracker_config,
        persist=True  # Maintains IDs across frames
    )

    # Draw bounding boxes and IDs
    if results and hasattr(results[0], 'boxes'):
        for box in results[0].boxes:
            # Check if box.id is not None before accessing it
            if box.id is not None:
                x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
                track_id = int(box.id)
                conf = float(box.conf)
                # Draw box and ID
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
                cv2.putText(frame, f'ID:{track_id}', (x1, y1-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

    # Show frame and save to output
    #cv2.imshow('Player Tracking', frame)
    out.write(frame)
    #if cv2.waitKey(1) & 0xFF == ord('q'):
    #    break

cap.release()
out.release()
#cv2.destroyAllWindows()


0: 384x640 16 players, 1 referee, 2899.6ms
Speed: 3.0ms preprocess, 2899.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 players, 2 referees, 2384.3ms
Speed: 3.1ms preprocess, 2384.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 16 players, 2 referees, 2432.8ms
Speed: 3.7ms preprocess, 2432.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 players, 2 referees, 2379.4ms
Speed: 4.0ms preprocess, 2379.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 players, 2 referees, 3337.9ms
Speed: 3.0ms preprocess, 3337.9ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 players, 2 referees, 2531.5ms
Speed: 5.2ms preprocess, 2531.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 players, 1 referee, 2405.7ms
Speed: 3.1ms preprocess, 2405.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384,

In [7]:
!pip install -U ultralytics